# Human Population Size on an Earth-like Planet--a Computer Experiment

In [259]:
from pathlib import Path

import networkx as nx
from pyvis.network import Network
import numpy as np
from scipy import stats
from pert import PERT

The response variable is total population size after SIMULATION_YEARS.

In [2]:
NUM_WORLD_LOCATIONS = 100
SIMULATION_YEARS = 10000
INITIAL_POPULATION_PROPORTION = 0.0001
# square miles 
TOTAL_LAND_AREA = 57268900 
# The average maximum sustainable population density 
# (per square mile) on the planet for a hunter-gatherer society.
# https://en.wikipedia.org/wiki/Hunter-gatherer#:~:text=One%20group%2C%20the%20Chumash%2C%20had,21.6%20persons%20per%20square%20mile.
INITIAL_MAX_POP_DENSITY = 20
INITIAL_AVG_CARRYING_CAPACITY_PER_LOCATION = (INITIAL_MAX_POP_DENSITY * TOTAL_LAND_AREA) / NUM_WORLD_LOCATIONS

print(
    f"INITIAL_AVG_CARRYING_CAPACITY_PER_LOCATION: {INITIAL_AVG_CARRYING_CAPACITY_PER_LOCATION}"
)

BIOMES = [
    "tropical and subtropical moist broadleaf forests",
    "tropical and subtropical dry broadleaf forests",
    "tropical and subtropical coniferous forests",
    "temperate broadleaf and mixed forests",
    "temperate coniferous forests",
    "boreal forests/taiga",
    "tropical and subtropical grasslands, savannas, and shrublands",
    "temperate grasslands, savannas, and shrublands",
    "flooded grasslands and savannas",
    "montane grasslands and shrublands",
    "tundra",
    "Mediterranean forests, woodlands, and scrub or sclerophyll forests",
    "deserts and xeric shrublands",
    "mangrove"
]

INITIAL_AVG_CARRYING_CAPACITY_PER_LOCATION: 11453780.0


In [3]:
biomes = Network(
    directed=False,
    neighborhood_highlight=False, 
    select_menu=True, 
    filter_menu=True,
    cdn_resources="in_line"
)

pre_biomes = nx.Graph()

# https://stackoverflow.com/a/47555011/8423001
nodes_and_biomes_dict = {node_id: BIOMES[node_id] for node_id in range(len(BIOMES))}

pre_biomes.add_nodes_from(
    [(node, {"biome": attribute}) 
        for (node, attribute) 
        in nodes_and_biomes_dict.items()
    ] 
)

pre_biomes.add_edges_from(
    [
        (0, 1),
        (0, 2),
        (0, 3),
        (0, 4),
        (0, 6),
        (0, 9),
        (0, 12),
        (0, 13),
        (1, 2),
        (1, 6),
        (1, 8),
        (1, 9),
        (1, 12),
        (1, 13),
        (2, 3),
        (2, 4),
        (2, 6),
        (2, 8),
        (2, 9),
        (2, 12),
        (2, 13),
        (3, 4),
        (3, 5),
        (3, 6),
        (3, 7),
        (3, 8),
        (3, 9),
        (3, 11),
        (3, 12),
        (3, 13),
        (4, 5),
        (4, 6),
        (4, 7),
        (4, 8),
        (4, 9),
        (4, 10),
        (4, 11),
        (4, 12),
        (4, 13),
        (5, 7),
        (5, 8),
        (5, 9),
        (5, 10),
        (6, 7),
        (6, 8),
        (6, 9),
        (6, 12),
        (6, 13),
        (7, 8),
        (7, 9),
        (7, 11),
        (7, 12),
        (8, 9),
        (8, 11),
        (8, 12),
        (8, 13),
        (9, 11),
        (9, 12),
        (9, 13),
        (11, 12),
        (11, 13),
        (12, 13)
    ]
)

In [4]:
# biomes.from_nx(nx_graph=pre_biomes)
# biomes.show("biomes.html")

In [5]:
world = Network(
    directed=True,
    neighborhood_highlight=True, 
    select_menu=True, 
    filter_menu=True,
    cdn_resources="in_line"
)

pre_world = nx.connected_watts_strogatz_graph(
    n=NUM_WORLD_LOCATIONS,
    k=5,
    p=0.5
)

In [6]:
for (v1, v2, weight) in pre_world.edges.data('weight'):
    # https://trenton3983.github.io/files/projects/2020-05-21_intro_to_network_analysis_in_python/2020-05-21_intro_to_network_analysis_in_python.html
    # https://stackoverflow.com/questions/40128692/networkx-how-to-add-weights-to-an-existing-g-edges

    # Here, the weights represent the ease of travelling between nodes.
    # A high weight indicates that travel is easy.
    pre_world[v1][v2]["weight"] = stats.expon.rvs(scale=1)

# Make the graph directed to indicate
# allowable population movements.
pre_world = pre_world.to_directed()

In [7]:
# Skip the last row because we only care about
# the upper triangle exclusive of th main diagonal
# of the adjacency matrix.
for v1 in range(NUM_WORLD_LOCATIONS - 1):
    for v2 in range(v1 + 1, NUM_WORLD_LOCATIONS):
        current_edge_data = pre_world.get_edge_data(v1, v2)
        if current_edge_data is None:
            # There is no need to update current_weight.
            continue

        # Extract weight attribute
        current_weight = current_edge_data["weight"]
        if current_weight < 1:
            # Make the ease of travel different
            # for one of the edges connecting the same
            # pair of nodes to simulate ocean currents.
            pre_world[v1][v2]["weight"] = stats.expon.rvs(scale=0.2)


In [8]:
pre_world_betweenness_centralities = nx.betweenness_centrality(
    G=pre_world,
    weight="weight"
)

pre_biomes_betweenness_centralities = nx.betweenness_centrality(
    G=pre_biomes
)

# Get a node with a maximal betweenness centrality.
# This node will hold our starting population.
# https://stackoverflow.com/a/280156/8423001
starting_node = max(
    pre_world_betweenness_centralities, 
    key=pre_world_betweenness_centralities.get
)

starting_node_biome_id = max(
    pre_biomes_betweenness_centralities, 
    key=pre_biomes_betweenness_centralities.get
)

starting_node_biome = BIOMES[starting_node_biome_id]

In [20]:
sorted(list(pre_biomes_betweenness_centralities.values()))

[0.0,
 0.0016025641025641025,
 0.0018315018315018313,
 0.005087505087505087,
 0.0066900691900691886,
 0.009523809523809525,
 0.016427553927553927,
 0.016427553927553927,
 0.02616503866503866,
 0.02849002849002849,
 0.033043345543345544,
 0.04137159137159137,
 0.05237586487586487,
 0.14557895807895804]

In [98]:
# https://stackoverflow.com/a/3071441/8423001
(
    stats.rankdata(
        a=list(pre_biomes_betweenness_centralities.values()),
        method="max"
    )
    # Because the ranks start at 1 but Python is 0-indexed,
    # subtract 1.
    - 1
)

array([ 3,  1,  4, 10, 13,  9,  7,  5, 11, 12,  0,  2,  8,  7])

In [23]:
sum(np.array(list(pre_biomes_betweenness_centralities.values())) <= 0.01)

6

In [106]:
stats.rankdata(
        a=[-2, 0, 3, 3, 3],
        method="max"
    )

array([1, 2, 5, 5, 5])

In [257]:
stats.binom.rvs(n=2, p=0.5, loc=10, size=100) - 1


array([10, 10, 10,  9, 10, 11, 10,  9, 10, 11, 10, 10,  9, 10, 11,  9,  9,
        9, 10, 10, 10, 11,  9, 11, 10, 10, 10, 10, 10, 11,  9, 10,  9, 11,
       10,  9, 10, 10, 11, 10, 10, 10, 10, 10,  9, 10, 10,  9, 10,  9,  9,
       10, 10, 10,  9,  9, 10, 10, 11, 10, 11, 10, 10, 11,  9,  9, 10, 10,
       10, 11, 10,  9, 10,  9, 10,  9,  9,  9, 10, 11, 11, 11,  9,  9, 10,
       10, 10,  9, 11, 11, 10, 11, 11, 10,  9, 11, 10,  9, 10, 11])

In [258]:
for i in range(1, 3):
    print(i)

1
2


In [262]:
def get_correlated_ranks(
    x_ranks, 
    num_y_obs
):
    """Given the ranks of observations for some variable X (x_ranks),
    return randomly generated ranks 
    for a desired number of observations of Y.

    The ranks for observations of Y are generated such that
    the Spearman rank correlation
    coefficient between X and Y is high.

    Args:
        x_ranks: list. This should be non-empty.
            The maximum element should be no larger than
            the length of the list.
        num_y_obs: integer.  This should be at least 2.

    Returns:
        list. The elements of the returned list correspond to 
            ranks for observations of Y.  The returned list
            should have a length of num_y_obs.
    """

    num_x_obs = len(x_ranks)
    y_ranks = []
    for x_rank in x_ranks:
        # Sample from a PERT distribution
        # with most likely value p_most_likely
        p_most_likely = (x_rank + 1) / (num_x_obs + 2)

        p = PERT(
            min_val=0,
            ml_val=p_most_likely,
            max_val=1
        ).rvs(size=1)
        # This binomial random variable can be anything from 0
        # to num_y_obs.  However, y_ranks should be anything
        # between 1 and num_y_obs.
        # The minus 1 ensures us that 
        y_ranks.append(stats.binom.rvs(n=num_y_obs - 1, p=p, size=1).item() + 1)
    
    return y_ranks

In [284]:
thingys = get_correlated_ranks(x_ranks=[3, 1.5, 1.5, 4.5, 4.5, 6], num_y_obs=100)
thingys

[40, 24, 12, 73, 92, 86]

In [44]:
# We plan on assigning biomes to the nodes in our world.
# But, we must consider that some biomes are more likely
# to be connected.  Thus, we assign the biomes randomly
# while taking account of the betweenness centralities.
# With probability 0.5, we assign neighbors the same
# biome, while with probability 0.5, we assign neighbors
# a new biome of similar betweenness centrality.
1.0 / NUM_WORLD_LOCATIONS
sorted(pre_biomes_betweenness_centralities.values())
# Given a value of the ECDF of pre_world_betweenness_centralities
# generate an appropriately positioned random rank
# within pre_biomes_betweenness_centralities.
# First, rank the pre_world_betweenness_centralities.
sorted(pre_world_betweenness_centralities.values())
# Second, find the find the value of the ECDF for each rank.

[0.0,
 0.0029185009086060416,
 0.0037909003431576094,
 0.004687778837438701,
 0.005389986682503692,
 0.0061083788974945425,
 0.006120787753440813,
 0.006147235228867882,
 0.0066454142984755235,
 0.006786613372884244,
 0.006825053253624683,
 0.006903177218576103,
 0.006908970344344496,
 0.006998715455734631,
 0.008223694695556168,
 0.008317136378360868,
 0.008683439032851526,
 0.009409518423123861,
 0.00942326873062804,
 0.010669472651538205,
 0.011048955854150657,
 0.011547843690700828,
 0.011826885636409446,
 0.01186854557293642,
 0.012032453219837265,
 0.01232521572657627,
 0.013003782561605687,
 0.013004273378423036,
 0.013099895706574743,
 0.013286589724437588,
 0.01403749426322092,
 0.014635421268074324,
 0.01525476516200078,
 0.015370910268869449,
 0.016635280704544772,
 0.01709491177642074,
 0.01720553776830832,
 0.01764910803253413,
 0.017986562327935236,
 0.018161114064020673,
 0.019257725828535972,
 0.019337906876558706,
 0.019343233783555364,
 0.019583749469340068,
 0.019757

In [36]:
np.quantile(
    a=list(pre_world_betweenness_centralities.values()),
    q=0.5
)

0.02331390236461356

In [29]:
for id in pre_biomes.neighbors(starting_node_biome_id):
    print(id)

0
2
3
5
6
7
8
9
10
11
12
13


In [16]:
# Loop through nodes and set initial parameters.
for node in nx.nodes(G=pre_world):
    nx.set_node_attributes(
        G=pre_world, 
        values={node: {"carrying_capacity": 1000000}}
    )

In [17]:
world.from_nx(nx_graph=pre_world, show_edge_weights=True)
world.show("world.html")